In [57]:
import os

import numpy as np
import pandas as pd

from Configurations.config import Config, config
from Utils.verbose_logger import v_logger

from Utils.dataframe_funcs import MedianDedupedDF

v_logger.UpdateFormat("PubChem_toxicity", "fg #FAC11C")

toxicity_config: Config = config["PubChem_download_toxicity"]

filtering_folder_name: str = f"{toxicity_config["results_folder_name"]}/filtering"
os.makedirs(filtering_folder_name, exist_ok=True)

# KG

In [58]:
kg_df = pd.read_csv(f"{toxicity_config["results_folder_name"]}/{toxicity_config["combined_file_name"]}_kg.csv",
sep=config["csv_separator"], low_memory=False)

unique_organisms = kg_df["organism"].unique()
v_logger.info(f"Unique organism: {unique_organisms}")

unique_routes = kg_df["route"].unique()
v_logger.info(f"Unique route: {unique_routes}")

kg_df

[14.03.2025 13:48:05] PubChem_toxicity: Unique organism: ['mouse' 'rat']                                              [INFO]
[14.03.2025 13:48:05] PubChem_toxicity: Unique route: ['subcutaneous' 'intraperitoneal' 'oral' 'intravenous']         [INFO]


,cid,sid,sourceid,organism,testtype,route,reference,mw,dose,dose_units,pLD50,effect,time_period
0,19081,134984244,3548105,mouse,LD50,subcutaneous,"British Journal of Experimental Pathology., 28...",208.26,130.0,mg/kg,6.204663,NaN,NaN
1,19083,134984246,3553808,mouse,LD50,subcutaneous,Australian Journal of Experimental Biology and...,145.20,680.0,mg/kg,5.329458,BEHAVIORAL: SOMNOLENCE (GENERAL DEPRESSED ACTI...,NaN
2,19239,134984251,3619178,mouse,LD50,intraperitoneal,Journal of Medicinal and Pharmaceutical Chemis...,102.14,767.0,mg/kg,5.124400,NaN,NaN
3,19240,134984252,3622762,rat,LD50,oral,"Toxicology and Applied Pharmacology., 28(313),...",115.17,180.0,mg/kg,5.806067,NaN,NaN
4,19242,134984253,3623050,mouse,LDLo,subcutaneous,Archiv fuer Experimentelle Pathologie und Phar...,127.18,230.0,mg/kg,5.742691,BEHAVIORAL: CONVULSIONS OR EFFECT ON SEIZURE T...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108623,3078298,135225175,120602280,mouse,LD50,intraperitoneal,Acta Poloniae Pharmaceutica. For English tran...,214.25,375.0,mg/kg,5.756890,NaN,NaN
108624,16684679,135225176,120626860,mouse,LD50,intraperitoneal,European Journal of Medicinal Chemistry--Chimi...,332.10,1000.0,mg/kg,5.521269,NaN,NaN
108625,16684680,135225177,120626871,mouse,LD50,intraperitoneal,European Journal of Medicinal Chemistry--Chimi...,281.94,800.0,mg/kg,5.547067,NaN,NaN
108626,3078453,135225178,121221054,mouse,LD50,intraperitoneal,"Pharmaceutical Chemistry Journal, 22(353), 1988",444.50,103.0,mg/kg,6.635035,NaN,NaN


In [59]:
unique_separated: dict[str, dict[str, pd.DataFrame]] = {}

for u_organism, organism_df in kg_df.groupby("organism"):
    unique_separated.setdefault(str(u_organism), {})

    for u_route, route_df in organism_df.groupby("route"):
        unique_separated[str(u_organism)][str(u_route)] = route_df.copy()

In [60]:
unique_test_types = kg_df["testtype"].unique()
v_logger.info(unique_test_types)

unique_filtered: dict[str, dict[str, dict[str, pd.DataFrame]]] = {}

for u_organism in unique_organisms:
  v_logger.info()
  v_logger.info(f"organism: {u_organism}")

  unique_filtered.setdefault(u_organism, {})

  for u_route in unique_routes:
    v_logger.info(f"    route: {u_route}")

    unique_filtered[u_organism].setdefault(u_route, {})

    df = unique_separated[u_organism][u_route]

    for u_test_type in unique_test_types:
      filtered_df = df[df["testtype"] == u_test_type].copy()
      filtered_df = MedianDedupedDF(filtered_df, "sid", "pLD50")

      if len(filtered_df) >= 100:
        unique_filtered[u_organism][u_route][u_test_type] = filtered_df

        os.makedirs(f"{filtering_folder_name}/kg", exist_ok=True)
        filtered_df.to_csv(f"{filtering_folder_name}/kg/{toxicity_config["results_file_name"]}_{u_organism}_{u_route}_{u_test_type}.csv")

        v_logger.info(f"      test_type: {u_test_type}, len: {len(unique_filtered[u_organism][u_route][u_test_type])}")

[14.03.2025 13:48:06] PubChem_toxicity: ['LD50' 'LDLo' 'LD20' 'LD10' 'LD70' 'LD80' 'LD40' 'LD60' 'LD95' 'LD16'
 'LD05' 'LD25' 'LD75' 'LD17' 'TDLo' 'LD90' 'LD30' 'LD55' 'LD12' 'LD54'] [INFO]
[14.03.2025 13:48:06] PubChem_toxicity: ----------------------------------------------------------------------------- [INFO]
[14.03.2025 13:48:06] PubChem_toxicity: organism: mouse                                                               [INFO]
[14.03.2025 13:48:06] PubChem_toxicity:     route: subcutaneous                                                       [INFO]
[14.03.2025 13:48:14] PubChem_toxicity:       test_type: LD50, len: 7039                                              [INFO]
[14.03.2025 13:48:15] PubChem_toxicity:       test_type: LDLo, len: 983                                               [INFO]
[14.03.2025 13:48:15] PubChem_toxicity:     route: intraperitoneal                                                    [INFO]
[14.03.2025 13:48:58] PubChem_toxicity:       test_type: LD5

# M3

In [61]:
m3_df = pd.read_csv(f"{toxicity_config["results_folder_name"]}/{toxicity_config["combined_file_name"]}_m3.csv",
sep=config["csv_separator"], low_memory=False)

unique_organisms = m3_df["organism"].unique()
v_logger.info(f"Unique organism: {unique_organisms}")

m3_df["time_period"] = m3_df["time_period"].replace(np.nan, "no_exact_time")

unique_time_periods = m3_df["time_period"].unique()
v_logger.info(f"Unique time_period: {unique_time_periods}")

m3_df

[14.03.2025 13:50:17] PubChem_toxicity: Unique organism: ['rat' 'mouse']                                              [INFO]


[14.03.2025 13:50:17] PubChem_toxicity: Unique time_period: ['4H' '10M' '2H' '6H' '5H' 'no_exact_time' '8H' '1H' '15M' '30M' '9M' '7H'
 '3H' '5.5H' '24H' '5M' '13M' '30S' '90' '41M' '2.3H' '45M' '3M' '4.5H'
 '22H' '30' '14H' '2M' '18H' '1M' '2.7H' '20M' '9H' '15' '80M'] [INFO]


,cid,sid,sourceid,organism,testtype,route,reference,mw,dose,dose_units,time_period,pLD50,effect
0,19240,134984252,3622762,rat,LCLo,inhalation,"Toxicology and Applied Pharmacology., 28(313),...",115.17,1.061474e+02,mg/m3,4H,6.035430,NaN
1,19756,134984284,3878191,rat,LC50,inhalation,Wirksubstanzen der Pflanzenschutz und Schadlin...,184.19,3.300000e+02,mg/m3,4H,5.746752,NaN
2,19950,134984301,4035896,mouse,LC50,inhalation,"Toxicology and Applied Pharmacology., 64(423),...",478.60,9.100000e+01,mg/m3,4H,6.720931,"LUNGS, THORAX, OR RESPIRATION: RESPIRATORY DEP..."
3,20972,134984357,4835114,rat,LC50,inhalation,"Food and Chemical Toxicology., 22(425), 1984 [...",228.42,2.200000e+02,mg/m3,4H,6.016311,SENSE ORGANS AND SPECIAL SENSES: OTHER CHANGES...
4,19092,134984416,3563368,rat,LC50,inhalation,"Toxicology and Applied Pharmacology., 5(677), ...",219.20,1.100000e+01,mg/m3,10M,7.299448,"BEHAVIORAL: ATAXIA; LUNGS, THORAX, OR RESPIRAT..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,20965,134984194,4824786,rat,LCLo,inhalation,Wirksubstanzen der Pflanzenschutz und Schadlin...,394.00,1.030127e+00,mg/m3,2H,8.582605,NaN
1742,3672772,135208215,1338234,rat,LC50,inhalation,American Industrial Hygiene Association Journa...,210.22,2.326135e+01,mg/m3,4H,6.956039,"LUNGS, THORAX, OR RESPIRATION: DYSPNEA"
1743,3672772,135208215,1338234,mouse,LC50,inhalation,American Industrial Hygiene Association Journa...,210.22,1.977214e+01,mg/m3,4H,7.026620,"LUNGS, THORAX, OR RESPIRATION: DYSPNEA"
1744,168826,135211825,37360988,rat,LC50,inhalation,"National Technical Information Service., 0TS05...",300.04,6.600000e+06,mg/m3,4H,1.657635,BEHAVIORAL: CONVULSIONS OR EFFECT ON SEIZURE T...


In [62]:
unique_separated: dict[str, dict[str, pd.DataFrame]] = {}

for u_organism, organism_df in m3_df.groupby("organism"):
    unique_separated.setdefault(str(u_organism), {})

    for u_time_period, time_period_df in organism_df.groupby("time_period"):
        unique_separated[str(u_organism)][str(u_time_period)] = time_period_df.copy()

In [63]:
unique_test_types = m3_df["testtype"].unique()
v_logger.info(unique_test_types)

unique_filtered: dict[str, dict[str, dict[str, pd.DataFrame]]] = {}

for u_organism in unique_organisms:
  v_logger.info()
  v_logger.info(f"organism: {u_organism}")

  unique_filtered.setdefault(u_organism, {})

  for u_time_period in unique_time_periods:
    v_logger.info(f"    time_period: {u_time_period}")

    unique_filtered[u_organism].setdefault(u_time_period, {})

    df: pd.DataFrame
    try:
      df = unique_separated[u_organism][u_time_period]
    except KeyError:
      continue

    for u_test_type in unique_test_types:
      filtered_df = df[df["testtype"] == u_test_type].copy()
      filtered_df = MedianDedupedDF(filtered_df, "sid", "pLD50")

      if len(filtered_df) >= 100:
        unique_filtered[u_organism][u_time_period][u_test_type] = filtered_df

        os.makedirs(f"{filtering_folder_name}/m3", exist_ok=True)
        filtered_df.to_csv(f"{filtering_folder_name}/m3/{toxicity_config["results_file_name"]}_{u_organism}_{u_time_period}_{u_test_type}.csv")

        v_logger.info(f"      test_type: {u_test_type}, len: {len(unique_filtered[u_organism][u_time_period][u_test_type])}")

[14.03.2025 13:50:18] PubChem_toxicity: ['LCLo' 'LC50' 'LC33' 'LD50' 'LC75']                                          [INFO]
[14.03.2025 13:50:18] PubChem_toxicity: ----------------------------------------------------------------------------- [INFO]
[14.03.2025 13:50:18] PubChem_toxicity: organism: rat                                                                 [INFO]
[14.03.2025 13:50:18] PubChem_toxicity:     time_period: 4H                                                           [INFO]
[14.03.2025 13:50:18] PubChem_toxicity:       test_type: LCLo, len: 262                                               [INFO]
[14.03.2025 13:50:18] PubChem_toxicity:       test_type: LC50, len: 354                                               [INFO]
[14.03.2025 13:50:18] PubChem_toxicity:     time_period: 10M                                                          [INFO]
[14.03.2025 13:50:18] PubChem_toxicity:     time_period: 2H                                                           [INFO]
